In [1]:
# TODO move to test file
from copy import deepcopy
from src.sim.scenario import Scenario
from src.trades import Swap
import cProfile
import pstats

In [2]:
scenario = Scenario("baseline", "weth")
sample = scenario.pricepaths[0]
scenario.prepare_for_run()
scenario.prepare_for_trades(sample)

[INFO][18:38:31][src.sim.scenario]-15980: Fetching sim_market from subgraph.
[INFO][18:38:43][src.utils.poolgraph]-15980: Found 20 valid trading cycles of length 3.


In [3]:
llamma = scenario.llamma
i = 0
j = 1
size = llamma.get_max_trade_size(i, j)

swap = Swap(llamma, i, j, None)

with llamma.use_snapshot_context():
    _, out_snapshot, _ = llamma.trade(i, j, size)

before = llamma.get_snapshot()
out_stateless, _ = swap.execute(size, use_snapshot_context=True)
after = llamma.get_snapshot()

assert out_snapshot == out_stateless

for (k_before, v_before), (k_after, v_after) in zip(
    before.__dict__.items(), after.__dict__.items()
):
    assert k_before == k_after
    if hasattr(v_before, "__dict__"):
        assert v_before.__dict__ == v_after.__dict__

In [4]:
spool = scenario.stableswap_pools[0]
i = 0
j = 1
size = spool.get_max_trade_size(i, j)

swap = Swap(spool, i, j, None)

with spool.use_snapshot_context():
    _, out_snapshot, _ = spool.trade(i, j, size)

before = spool.get_snapshot()
out_stateless, _ = swap.execute(size, use_snapshot_context=True)
after = spool.get_snapshot()

assert out_snapshot == out_stateless

for (k_before, v_before), (k_after, v_after) in zip(
    before.__dict__.items(), after.__dict__.items()
):
    assert k_before == k_after
    if hasattr(v_before, "__dict__"):
        assert v_before.__dict__ == v_after.__dict__

In [5]:
pool = deepcopy(scenario.stableswap_pools[0])
i = 0
j = 1
amt_in = pool.get_max_trade_size(i, j)

swap = Swap(pool, i, j, None)

amt_out_stateless, _ = swap.execute(amt_in, use_snapshot_context=True)

swap.amt = amt_in
amt_out_statefull, _ = swap.execute(amt_in, use_snapshot_context=False)

assert amt_out_stateless == amt_out_statefull

In [7]:
pool = deepcopy(scenario.llamma)
i = 0
j = 1
amt_in = pool.get_max_trade_size(i, j)

swap = Swap(pool, i, j, None)

amt_out_stateless, _ = swap.execute(amt_in, use_snapshot_context=True)

swap.amt = amt_in
amt_out_statefull, _ = swap.execute(amt_in, use_snapshot_context=False)

assert amt_out_stateless == amt_out_statefull

In [9]:
pool = list(scenario.markets.values())[0]
i = 0
j = 1
amt_in = pool.get_max_trade_size(i, j)

swap = Swap(pool, i, j, None)

amt_out_stateless, _ = swap.execute(amt_in, use_snapshot_context=True)

swap.amt = amt_in
amt_out_statefull, _ = swap.execute(amt_in, use_snapshot_context=False)

assert amt_out_stateless == amt_out_statefull